**NOTE:** The below implementation is highly inspired from he PINN paper from 2019 and their open source repository, the data and model architecture has been referenced from there.

In [39]:
# importing the libraries

import tensorflow as tf
from tensorflow.keras import layers, Model
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.io
from pyDOE import lhs

import time
import tqdm
import pprint

### Data Preparation

In [55]:
# Domain boundaries

x_min, x_max = -5.0, 5.0
t_min, t_max = 0.0, np.pi/2

lb = np.array([x_min, t_min])
ub = np.array([x_max, t_max])

# lb - lower bound, ub - upper bound

N0 = 20
N_b = 20
N_f = 10000

# N0 - number of points in the initial condition, 
# N_b - number of points in the boundary condition, 
# N_f - number of points in the interior domain

layers = [2, 50, 50, 50, 50, 2]

In [56]:
data = scipy.io.loadmat('../data/NLS.mat')

In [57]:
list(data.keys())

['__header__', '__version__', '__globals__', 'tt', 'uu', 'x']

`tt` - Time Grid

`uu` - Full Solution Array

`x` - Spatial Grid

In [58]:
print(f"Shape of time grid: {data['tt'].shape}")
print(f"Shape of solution array: {data['uu'].shape}")
print(f"Shape of spatial grid: {data['x'].shape}")


Shape of time grid: (1, 201)
Shape of solution array: (256, 201)
Shape of spatial grid: (1, 256)


In [59]:
t = data['tt'].flatten()[:, None]
x = data['x'].flatten()[:, None]
Exact = data['uu']
Exact_u = np.real(Exact)
Exact_v = np.imag(Exact)

Exact_h_modulus = np.sqrt(Exact_u**2 + Exact_v**2)

# h = u + iv
# h_modulus = sqrt(u^2 + v^2)

In [60]:
print(f"Shape of exact solution: {Exact.shape}")
print(f"Shape of time grid: {t.shape}")
print(f"Shape of spatial grid: {x.shape}")


Shape of exact solution: (256, 201)
Shape of time grid: (201, 1)
Shape of spatial grid: (256, 1)


In [61]:
X, T = np.meshgrid(x, t)

X_star = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))

u_star = Exact_u.flatten()[:, None]
v_star = Exact_v.flatten()[:, None]
h_star = Exact_h_modulus.flatten()[:, None]

print(f"Shape of u_star: {u_star.shape}")
print(f"Shape of v_star: {v_star.shape}")
print(f"Shape of h_star: {h_star.shape}")
print(f"Shape of X_star: {X_star.shape}")

# X_star - spatial and temporal grid
# u_star - real part of the solution
# v_star - imaginary part of the solution
# h_star - modulus of the solution

Shape of u_star: (51456, 1)
Shape of v_star: (51456, 1)
Shape of h_star: (51456, 1)
Shape of X_star: (51456, 2)


In [62]:
idx_x = np.random.choice(x.shape[0], N0, replace=False)

x0 = x[idx_x,:]
u0 = Exact_u[idx_x,0:1]
v0 = Exact_v[idx_x,0:1]

idx_t = np.random.choice(t.shape[0], N_b, replace=False)
t_b = t[idx_t,:]

X_f = lb + (ub - lb) * lhs(2, N_f)

# x0 - selected x points from the initial condition where t = 0
# u0 - real part of the initial condition
# v0 - imaginary part of the initial condition


# t_b - time values at which the spatial boundaries will be sampled.
# X_f - interior points (xf, tf) used to enforce the PDE via the residuals f_u and f_v.


`lhs(2, N_f)`: Latin Hypercube Sampling of 2D space ([0,1]^2) to get N_f points.

| Set                       | How sampled                     | Domain             | Purpose                           |
| ------------------------- | ------------------------------- | ------------------ | --------------------------------- |
| `x0, u0, v0`              | random `N0` spatial points      | t=0                | **initial condition**             |
| `tb` (used in X_lb, X_ub) | random `Nb` time points         | x=lb, ub           | **boundary condition**            |
| `X_f`                     | `Nf` points via Latin Hypercube | interior 2D domain | **collocation / PDE enforcement** |


Now that the data preparation is complete, we move on to **modeling the neural network**. 

- The datasets `x0`, `u0`, `v0`, `tb`, and `X_f` are used to **compute the PINN’s loss function** during training:

  - **Initial condition points** (`x0`, `u0`, `v0`):  
    These are spatial points at \(t=0\) and enforce the initial state of the system.

  - **Boundary condition points** (`tb`, `lb`, `ub`):  
    These correspond to time points at the spatial boundaries and enforce constraints at the edges of the domain.

  - **Collocation points** (`X_f`):  
    These are points in the interior of the domain where the PDE residuals are enforced, ensuring the neural network satisfies the governing equations.

- The `_star` datasets (e.g., `X_star`, `u_star`, `v_star`, `h_star`) represent the **full reference solution on a dense grid**, which is used for **prediction, evaluation, and error computation** after training. These points are **not used in computing the loss**, but serve as a benchmark to assess the accuracy of the network’s predictions.

**Summary:**  
- `x0, u0, v0, tb, X_f` → **training points** used to enforce physics, initial, and boundary conditions.  
- `_star` values → **evaluation points** used to validate the model’s performance.

In [86]:
# creating the PINN model

class PhysicsInformedNN(tf.keras.Model):
    # Initialize the PINN model
    def __init__(self, x0, u0, v0, tb, lb, ub, X_f, layers, lr):
        """
        Initialize the PhysicsInformedNN model.

        Args:
            x0 (np.ndarray): Initial spatial points
            u0 (np.ndarray): Initial real part of solution
            v0 (np.ndarray): Initial imaginary part of solution
            tb (np.ndarray): Boundary time points
            lb (np.ndarray): Lower spatial boundary
            ub (np.ndarray): Upper spatial boundary
            X_f (np.ndarray): Collocation points
            layers (list): List of layer sizes for the neural network
            lr (float): Learning rate for the optimizer
        """  
        super(PhysicsInformedNN, self).__init__()
        self.layer_sizes = layers
        self.lr = lr
        
        # what are we predicting?
        # u,v at initial condition (u0, v0)
        # u,v at boundary (u_lb, v_lb, u_ub, v_ub)
        # f_u, f_v at interior points (f_u_int, f_v_int)
        
        # constructing input data for the model for all the above conditions

        X0 = np.concatenate((x0, np.zeros_like(x0)), 1) # (x0, t=0)
        X_lb = np.concatenate((lb[0]*np.ones_like(tb), tb), 1) # (x=lb, t)
        X_ub = np.concatenate((ub[0]*np.ones_like(tb), tb), 1) # (x=ub, t)
        
        self.lb = tf.constant(lb, dtype=tf.float32)
        self.ub = tf.constant(ub, dtype=tf.float32)
        self.u0 = tf.constant(u0, dtype=tf.float32)
        self.v0 = tf.constant(v0, dtype=tf.float32)
        
        self.x0 = tf.constant(X0[:,0:1], dtype=tf.float32)
        self.t0 = tf.constant(X0[:,1:2], dtype=tf.float32)
        self.x_lb = tf.constant(X_lb[:,0:1], dtype=tf.float32)
        self.t_lb = tf.constant(X_lb[:,1:2], dtype=tf.float32)
        self.x_ub = tf.constant(X_ub[:,0:1], dtype=tf.float32)
        self.t_ub = tf.constant(X_ub[:,1:2], dtype=tf.float32)

        self.x_f = tf.constant(X_f[:,0:1], dtype=tf.float32)
        self.t_f = tf.constant(X_f[:,1:2], dtype=tf.float32)

        self.hidden_layers = [tf.keras.layers.Dense(layer, activation='tanh') for layer in self.layer_sizes[1:-1]]
        self.output_layer = tf.keras.layers.Dense(self.layer_sizes[-1])

        self.optimizer = tf.optimizers.Adam(self.lr)
        
        # Build the model by calling it with dummy data to initialize trainable_variables
        dummy_input = tf.zeros((1, 2), dtype=tf.float32)
        _ = self.call(dummy_input)

        
    def call(self, X):
        """Forward pass through the neural network."""
        H = 2.0 * (X - self.lb) / (self.ub - self.lb) - 1.0
        for layer in self.hidden_layers:
            H = layer(H)

        Y = self.output_layer(H)

        return Y

    def net_uv(self, x, t):
        """Get u, v and their spatial derivatives."""
        # Create a single variable for the input
        X = tf.concat([x, t], 1)
        X_var = tf.Variable(X, dtype=tf.float32)

        with tf.GradientTape(persistent=True) as tape:
            tape.watch(X_var)
            uv = self.call(X_var)
            u = uv[:, 0:1]
            v = uv[:, 1:2]

        # Compute first derivatives with respect to the full input
        u_grad = tape.gradient(u, X_var)
        v_grad = tape.gradient(v, X_var)

        # Extract spatial and temporal derivatives
        u_x = u_grad[:, 0:1] if u_grad is not None else tf.zeros_like(x)
        v_x = v_grad[:, 0:1] if v_grad is not None else tf.zeros_like(x)
        u_t = u_grad[:, 1:2] if u_grad is not None else tf.zeros_like(t)
        v_t = v_grad[:, 1:2] if v_grad is not None else tf.zeros_like(t)

        # Compute second derivatives
        u_xx = tape.gradient(u_x, X_var)
        v_xx = tape.gradient(v_x, X_var)
        
        # Extract second spatial derivatives
        u_xx = u_xx[:, 0:1] if u_xx is not None else tf.zeros_like(x)
        v_xx = v_xx[:, 0:1] if v_xx is not None else tf.zeros_like(x)

        del tape

        return u, v, u_x, v_x, u_t, v_t, u_xx, v_xx

    def net_f_uv(self, x, t):
        """Compute PDE residuals."""
        u, v, u_x, v_x, u_t, v_t, u_xx, v_xx = self.net_uv(x, t)

        f_u = u_t + 0.5*v_xx + (u**2 + v**2)*v
        f_v = v_t - 0.5*u_xx - (u**2 + v**2)*u

        return f_u, f_v
    
    def get_trainable_variables_info(self):
        """Get information about trainable variables."""
        print(f"Number of trainable variables: {len(self.trainable_variables)}")
        for i, var in enumerate(self.trainable_variables):
            print(f"Variable {i}: shape {var.shape}, name {var.name}")
        return self.trainable_variables
    

    
    def train(self, iterations):
        """Training loop for the PINN model."""

        start_time = time.time()
        for i in range(iterations):
            with tf.GradientTape() as tape:
                # forward pass
                u0_pred, v0_pred, u0_x, v0_x, u0_t, v0_t, u0_xx, v0_xx = self.net_uv(self.x0, self.t0)
                u_lb_pred, v_lb_pred, u_lb_x, v_lb_x, u_lb_t, v_lb_t, u_lb_xx, v_lb_xx = self.net_uv(self.x_lb, self.t_lb)
                u_ub_pred, v_ub_pred, u_ub_x, v_ub_x, u_ub_t, v_ub_t, u_ub_xx, v_ub_xx = self.net_uv(self.x_ub, self.t_ub)
                f_u_pred, f_v_pred = self.net_f_uv(self.x_f, self.t_f)

                # compute loss
                loss_u0 = tf.reduce_mean(tf.square(u0_pred - self.u0))
                loss_v0 = tf.reduce_mean(tf.square(v0_pred - self.v0))
                loss_bc = tf.reduce_mean(tf.square(u_lb_pred - u_ub_pred)) + tf.reduce_mean(tf.square(v_lb_pred - v_ub_pred))
                loss_bc_x = tf.reduce_mean(tf.square(u_lb_x - u_ub_x)) + tf.reduce_mean(tf.square(v_lb_x - v_ub_x))
                loss_f = tf.reduce_mean(tf.square(f_u_pred)) + tf.reduce_mean(tf.square(f_v_pred))

                loss = loss_u0 + loss_v0 + loss_bc + loss_bc_x + loss_f
            
            grads = tape.gradient(loss, self.trainable_variables)
            self.optimizer.apply_gradients(zip(grads, self.trainable_variables))

            if i % 2 == 0:
                elapsed = time.time() - start_time
                tqdm.tqdm.write(f"Iteration {i}, Loss: {loss.numpy()}, Time: {elapsed}")
                start_time = time.time()
                try:
                    print(f"Number of trainable variables: {len(self.trainable_variables)}")
                    print(f"Trainable variables shapes: {[var.shape for var in self.trainable_variables[:5]]}")  # Show first 5
                except Exception as e:
                    print(f"Could not access trainable variables: {e}")

In [87]:
lr = 0.001
model = PhysicsInformedNN(x0, u0, v0, t_b, lb, ub, X_f, layers, lr)

In [88]:
iterations = 10
model.train(iterations)

Iteration 0, Loss: 0.7512832283973694, Time: 0.18286609649658203
Number of trainable variables: 10
Trainable variables shapes: [TensorShape([2, 50]), TensorShape([50]), TensorShape([50, 50]), TensorShape([50]), TensorShape([50, 50])]
Iteration 2, Loss: 0.4648878574371338, Time: 0.15547800064086914
Number of trainable variables: 10
Trainable variables shapes: [TensorShape([2, 50]), TensorShape([50]), TensorShape([50, 50]), TensorShape([50]), TensorShape([50, 50])]
Iteration 4, Loss: 0.5005298852920532, Time: 0.15757203102111816
Number of trainable variables: 10
Trainable variables shapes: [TensorShape([2, 50]), TensorShape([50]), TensorShape([50, 50]), TensorShape([50]), TensorShape([50, 50])]
Iteration 6, Loss: 0.41305243968963623, Time: 0.14287614822387695
Number of trainable variables: 10
Trainable variables shapes: [TensorShape([2, 50]), TensorShape([50]), TensorShape([50, 50]), TensorShape([50]), TensorShape([50, 50])]
Iteration 8, Loss: 0.37184715270996094, Time: 0.154271841049194